# Least Square Fit

The purpose of this notebook is to illustrate the method of applying that matrix inverse to input data is an efficient way of fitting parameters.

In [33]:
import numpy as np
import scipy as sp
import time
import scipy.linalg
import scipy.optimize

#begin declaring primary functions (matrix squareroot and a check function)
def matrixsqrt(V, label="0", returntime=False):
    start = time.time()
    N = len(V[0])
    wt = np.empty([N,N]) #square root matrix (transposed)
    logdet = 0.
    #extracts eigenvalues and eigenvectors (bottleneck!)
    eigs, eigvecs = sp.linalg.eigh(V)
    for i in range(N):
        #sets each column in our transposed square root to the eigenvalue scaled by 1/sqrt(eig)
        wt[:,i] = (1./np.sqrt(eigs[i])) * eigvecs[:,i]
        logdet += np.log(2 * np.pi * eigs[i])
        #transposes the result
    w = np.ndarray.transpose(wt)
    end = time.time()
    dt = end-start
    if(label!="0"):
        print("Time elapsed for " + label + " is: " + str(dt) + "s")
    if(returntime==True):
        return w, logdet, dt
    else:
        return w, logdet

def ismsqrt(w,V):
    N = len(V[0])
    if(np.allclose(np.dot(V, np.dot(np.ndarray.transpose(w), w)),np.identity(N))):
        return True
    else:
        return False
    
#variance matrix generators
def makeV2(N, sigmarange=[100,300]):
    sigma2 = np.random.rand(N)*(sigmarange[1]-sigmarange[0]) + sigmarange[0]
    V2 = sigma2 * np.identity(N)
    return V2

def makeV3(N, sigmarange=[100,300], A=300, tau=30):
    K = np.empty([N,N])
    t = np.arange(N, dtype='int')
    for i in range(N):
        for j in range(N):
            K[i][j] = A * np.exp(-0.5 * (t[i]-t[j])**2 / (tau **2))
    V3 = makeV2(N, sigmarange=sigmarange) + K
    return V3

#importing preferred model
def expf(x, a, b):
    return np.exp((a*(x-b)))

def linef(x, a, b):
    return a*x + b

def linef2 (x, x1, y1, x2, y2):
    m = (y2-y1)/(x2-x1)
    b = y2 - (m*x2)
    return ((m*x) + b)

def piecewise(x, params): # params are: xl, xr, a1, b1, a3, b3
        xl = params[0]
        xr = params[1]
        a1 = params[2]
        b1 = params[3]
        a3 = params[4]
        b3 = params[5]
        result = np.empty(len(x))
        for i in range(len(x)):
                if(x[i]<=xl):
                    result[i] = expf(x[i], a1, b1)
                elif (x[i]>xl and x[i]<=xr):
                    result[i] = linef2(x[i], xl, expf(xl,a1,b1), xr, expf(xr, a3, b3))
                elif (x[i]>xr):
                    result[i] = expf(x[i], a3, b3)
        return result


Let's see if we need to use $w$ or $w^T$:

In [44]:
#opening data
data1 = "/Users/chris/Documents/QPP/SolarFlareGPs/data/121022782_ctime_lc.txt"
t1, I1 = np.loadtxt(data1, unpack=True)
t1 = t1-t1[0] #start data at 0

#from prior fit, estimations of piecewise function + A and tau estimates
params = [300, 30, 1100, 1500, 1.3e-2, 100, -9e-3, 2900]

#check to use w over wT:
def checkw(params, I, t):
    V = makeV3(len(t), A=params[0], tau=params[1])
    w, logdet = matrixsqrt(V)
    model = piecewise(t, params[2:])
    resid = np.dot(w, (I - model))
    chisq = np.dot(resid, resid)
    resid2 = (I-model)
    Vinv = np.linalg.inv(V)
    chisq2 = np.dot(resid2, np.dot(Vinv,resid2))
    print chisq
    print chisq2
    print (chisq-chisq2)

#checkw(params, I1, t1)
#w is correct

def residualtest(params, I, t):
    return (I - piecewise(t, params[2:]))

#define residual function, using
def residualPW(paramsV, paramsPW, I, t):
    V = makeV3(len(t), A=paramsV[0], tau=paramsV[1])
    w, logdet = matrixsqrt(V)
    model = piecewise(t, paramsPW)
    return np.dot(w, (I - model))



opt, flag = sp.optimize.leastsq(residualtest, params, args = (I1,t1), maxfev = int(1e5))
print opt[2:]
params[2:] = opt[2:]
opt, flag = sp.optimize.leastsq(residualPW, params[:2], args = (params[2:], I1,t1), maxfev = int(1e5))
print opt

[  1.14032007e+03   2.98214625e+06  -9.19314915e-03  -1.49606147e+03
  -4.69222043e+01  -4.72141199e+06]
[ 300.   30.]
